In [1]:
from time import sleep, strftime
from random import randint
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
import smtplib
from email.mime.multipart import MIMEMultipart

In [3]:
chromedriver_path= '.\chromedriver.exe'

In [4]:
driver = webdriver.Chrome(executable_path=chromedriver_path)
sleep(2)

In [5]:
yatra='https://flight.yatra.com/air-search-ui/dom2/trigger?type=R&viewName=normal&flexi=0&noOfSegments=2&origin=DEL&originCountry=IN&destination=BOM&destinationCountry=IN&flight_depart_date=30%2F09%2F2022&arrivalDate=04%2F10%2F2022&ADT=1&CHD=0&INF=0&class=Economy&source=fresco-home&unqvaldesktop=456183416202'
cleartrip='https://www.cleartrip.com/flights/results?adults=3&childs=0&infants=0&class=Economy&depart_date=14/11/2022&return_date=18/11/2022&from=DEL&to=IXZ&intl=n&origin=DEL%20-%20New%20Delhi,%20IN&destination=IXZ%20-%20Port%20Blair,%20IN&sd=1660963642740&rnd_one=R&sourceCountry=New%20Delhi&destinationCountry=Port%20Blair'
kayak='https://www.kayak.co.in/flights/DEL-BOM/2022-09-21-flexible/2022-09-27-flexible?sort=bestflight_a'
driver.get(kayak)


In [6]:
# //*[@id="modifySearch"]/ul[2]/li[4]/div[1]/label/div/div[2]/div[2]/div[3]/div[21]/p/span[2]

In [7]:
pop_close='//*[@class="B5lA-button"]'
closetry='//*[@class="bBPb-closeIcon"]'
#driver.find_element_by_xpath(pop_close).click()
#driver.find_element_by_xpath(closetry).click()

In [8]:
def load_more():
    try:
        more_results='//*[@class="moreButton"]'
        driver.find_element_by_xpath(more_results).click()
        print('sleeping....')
        sleep(randint(25,35))
    except:
        pass

In [12]:
def page_scrape():
    """This function takes care of the scraping part"""
    
    xp_sections = '//*[@class="section duration"]'
    sections = driver.find_elements_by_xpath(xp_sections)
    sections_list = [value.text for value in sections]
    section_a_list = sections_list[::2] # This is to separate the two flights
    section_b_list = sections_list[1::2] # This is to separate the two flights
    
    # if you run into a reCaptcha, you might want to do something about it
    # you will know there's a problem if the lists above are empty
    # this if statement lets you exit the bot or do something else
    # you can add a sleep here, to let you solve the captcha and continue scraping
    # i'm using a SystemExit because i want to test everything from the start
    if section_a_list == []:
        sleep(100)
    
    # I'll use the letter A for the outbound flight and B for the inbound
    a_duration = []
    a_section_names = []
    for n in section_a_list:
        # Separate the time from the cities
        a_section_names.append(''.join(n.split()[2:5]))
        a_duration.append(''.join(n.split()[0:2]))
    b_duration = []
    b_section_names = []
    for n in section_b_list:
        # Separate the time from the cities
        b_section_names.append(''.join(n.split()[2:5]))
        b_duration.append(''.join(n.split()[0:2]))

    xp_dates = '//div[@class="section date"]'
    dates = driver.find_elements_by_xpath(xp_dates)
    dates_list = [value.text for value in dates]
    a_date_list = dates_list[::2]
    b_date_list = dates_list[1::2]
    # Separating the weekday from the day
    a_day = [value.split()[0] for value in a_date_list]
    a_weekday = [value.split()[1] for value in a_date_list]
    b_day = [value.split()[0] for value in b_date_list]
    b_weekday = [value.split()[1] for value in b_date_list]
    
    # getting the prices
    xp_prices = '//a[@class="booking-link"]/span[@class="price option-text"]'
    prices = driver.find_elements_by_xpath(xp_prices)
    prices_list = [price.text.replace('$','') for price in prices if price.text != '']
    prices_list = list(map(int, prices_list))

    # the stops are a big list with one leg on the even index and second leg on odd index
    xp_stops = '//div[@class="section stops"]/div[1]'
    stops = driver.find_elements_by_xpath(xp_stops)
    stops_list = [stop.text[0].replace('n','0') for stop in stops]
    a_stop_list = stops_list[::2]
    b_stop_list = stops_list[1::2]

    xp_stops_cities = '//div[@class="section stops"]/div[2]'
    stops_cities = driver.find_elements_by_xpath(xp_stops_cities)
    stops_cities_list = [stop.text for stop in stops_cities]
    a_stop_name_list = stops_cities_list[::2]
    b_stop_name_list = stops_cities_list[1::2]
    
    # this part gets me the airline company and the departure and arrival times, for both legs
    xp_schedule = '//div[@class="section times"]'
    schedules = driver.find_elements_by_xpath(xp_schedule)
    hours_list = []
    carrier_list = []
    for schedule in schedules:
        hours_list.append(schedule.text.split('\n')[0])
        carrier_list.append(schedule.text.split('\n')[1])
    # split the hours and carriers, between a and b legs
    a_hours = hours_list[::2]
    a_carrier = carrier_list[::2]
    b_hours = hours_list[1::2]
    b_carrier = carrier_list[1::2]

    
    cols = (['Out Day', 'Out Time', 'Out Weekday', 'Out Airline', 'Out Cities', 'Out Duration', 'Out Stops', 'Out Stop Cities',
            'Return Day', 'Return Time', 'Return Weekday', 'Return Airline', 'Return Cities', 'Return Duration', 'Return Stops', 'Return Stop Cities',
            'Price'])

    flights_df = pd.DataFrame({'Out Day': a_day,
                               'Out Weekday': a_weekday,
                               'Out Duration': a_duration,
                               'Out Cities': a_section_names,
                               'Return Day': b_day,
                               'Return Weekday': b_weekday,
                               'Return Duration': b_duration,
                               'Return Cities': b_section_names,
                               'Out Stops': a_stop_list,
                               'Out Stop Cities': a_stop_name_list,
                               'Return Stops': b_stop_list,
                               'Return Stop Cities': b_stop_name_list,
                               'Out Time': a_hours,
                               'Out Airline': a_carrier,
                               'Return Time': b_hours,
                               'Return Airline': b_carrier,                           
                               'Price': prices_list})[cols]
    
    flights_df['timestamp'] = strftime("%Y%m%d-%H%M") # so we can know when it was scraped
    return flights_df
    

In [10]:
def start_yatra(from_city, to_city, date_start, date_end):
    kayak= ('https://www.kayak.co.in/flights/'+ from_city + '-' + to_city +'/'+ date_start + '-flexible/' + date_end + '-flexible?sort=bestflight_a')
    driver.get(kayak)
    sleep(randint(8,10))
    try:
        pop_close='//a[@class="B5lA-button"]'
        driver.find_element_by_xpath(pop_close).click()
    except Exception as e:
        pass
    sleep(randint(60,95))
    print('loading more...')
    load_more()
    
    #best
    
    print('starting scrape 1')
    df_best=page_scrape()
    df_best['sort']='best'
    sleep(randint(60,80))
    
    #lowest price
    matrix= driver.find_element_by_xpath('//*[contains(@id,"FlexMatrixCell")]')
    matrix_price=[price.text.replace('₹','') for price in matrix]
    matrix_price=list(map(int,matrix_price))
    matrix_min=min(matrix_price)
    matrix_avg= sum(matrix_price)/len(matrix_price)
    
    print('cheapest results')
    cheap='//a[@data-code="price"]'
    driver.find_element_by_xpath(cheap).click()
    sleep(randint(60,80))
    print('starting scrape 2')
    df_cheap=page_scrape()
    df_cheap['sort']='cheap'
    sleep(randint(60,80))
    
    #fastest
    
    print('fastest results')
    fast='//a[@data-code="duration"]'
    driver.find_element_by_xpath(fast).click()
    sleep(randint(60,80))
    print('starting scrape 3')
    df_fast=page_scrape()
    df_fast['sort']='fast'
    sleep(randint(60,80))
    
    df= df_cheap.append(df_best).append(df_fast)
    df.to_excel('search_backups//{}_flights_{}-{}_from_{}_to_{}.xlsx'.format(strftime("%Y%m%d-%H%M"),from_city, to_city, date_start, date_end), index=False)
    print('saved')
    
     

In [13]:
city_from = input('From which city? ')
city_to = input('Where to? ')
date_start = input('Search around which departure date? Please use YYYY-MM-DD format only ')
date_end = input('Return when? Please use YYYY-MM-DD format only ')

# city_from = 'LIS'
# city_to = 'SIN'
# date_start = '2019-08-21'
# date_end = '2019-09-07'

for n in range(0,5):
    start_yatra(city_from, city_to, date_start, date_end)
    print('iteration {} was complete @ {}'.format(n, strftime("%Y%m%d-%H%M")))
    
    # Wait 4 hours
    #sleep(60*60*4)
    print('sleep finished.....')

From which city? DEL
Where to? BOM
Search around which departure date? Please use YYYY-MM-DD format only 2022-09-21
Return when? Please use YYYY-MM-DD format only 2022-09-27
loading more...
sleeping....
starting scrape 1


ValueError: All arrays must be of the same length